<a href="https://colab.research.google.com/github/giu176/NetworkSecurity-ZKP/blob/main/ZKP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ZERO KNOWLEDGE PROOF

---





Project for Network Security and Cryptography
by Giulio Bosisio

The aim of the simulation is to perform a ZK-SNARK (Zero Knowledge - Succinct Non-Interactive Argument of Knowledge) computation of a python script. Succint means that the computation produce a proof that is both small in size and fast to verify. This proof is also constructed without interacting the verifier which makes the computation Non-Interactive, a classical Interactive protocol would have required the Verifier to challenge the Prover in real time. With a SNARK proof (a textual file) the Verifier can perform a verification whenever he wants without gaining any additional information about the witness (solution of the problem, the python script in this simulation) the verifier knows only the output of a specific input of the program (Zero Knowledge).

Properties of Zero Knowledge:
*   Completeness: If the Prover is honest, then it will eventually convince the Verifier.
*   Soundness: The Prover can only convince the Verifier if the statement is true.
*   Zero-knowledge(ness): The Verifier learns no information beyond the fact that the statement is true.



## Installing dependencies:


---



In [ ]:
!mkdir TrustedParty && mkdir Prover && mkdir Verifier
!pip3 install git+https://github.com/meilof/pysnark
!sudo apt install build-essential cmake git libgmp3-dev libprocps-dev python3-markdown libboost-program-options-dev libssl-dev python3 pkg-config

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/meilof/pysnark to /tmp/pip-req-build-9z49fes4
  Running command git clone -q https://github.com/meilof/pysnark /tmp/pip-req-build-9z49fes4
  Created wheel for PySNARK: filename=PySNARK-0.3.1-py3-none-any.whl size=116886 sha256=29a070cb442e31239a1d5d02d4934f7a38a8115e9966953506338cade865480d
  Stored in directory: /tmp/pip-ephem-wheel-cache-ud2ybqut/wheels/ee/ac/e0/e83d7fb143db3e217346b033cccb8100fecdac64bc9d9237ab
Successfully built PySNARK
Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
libboost-program-options-dev is already the newest version (1.65.1.0ubuntu1).
libboost-program-options-dev set to manually installed.
pkg-config is already the newest version (0.29.1-0ubuntu2).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.2).
git is alread

Default backend for pysnark:
```
pip3 install python-libsnark
```
libsnark backend doesn't work on Google Colab, so I used another backend ([qaptools](https://github.com/Charterhouse/qaptools)).

Building qaptools backend:

In [ ]:
!git clone --recursive https://github.com/Charterhouse/qaptools.git
!cd qaptools/ && mkdir build && cd build/ && cmake .. -DCURVE=ALT_BN128 -DUSE_PT_COMPRESSION=OFF -DWITH_PROCPS=OFF -DBINARY_OUTPUT=OFF && make && sudo make install

Cloning into 'qaptools'...
remote: Enumerating objects: 70, done.
remote: Total 70 (delta 0), reused 0 (delta 0), pack-reused 70
Unpacking objects: 100% (70/70), done.
Submodule 'depends/ate-pairing' (https://github.com/herumi/ate-pairing) registered for path 'depends/ate-pairing'
Submodule 'depends/xbyak' (https://github.com/herumi/xbyak) registered for path 'depends/xbyak'
Cloning into '/content/qaptools/depends/ate-pairing'...
remote: Enumerating objects: 1479, done.        
remote: Total 1479 (delta 0), reused 0 (delta 0), pack-reused 1479        
Receiving objects: 100% (1479/1479), 553.64 KiB | 13.84 MiB/s, done.
Resolving deltas: 100% (947/947), done.
Cloning into '/content/qaptools/depends/xbyak'...
remote: Enumerating objects: 6461, done.        
remote: Counting objects: 100% (1489/1489), done.        
remote: Compressing objects: 100% (438/438), done.        
remote: Total 6461 (delta 1086), reused 1348 (delta 1039), pack-reused 4972        
Receiving objects: 100% (6461/646

## Trusted Party


---



The example script perform a quadratic equation (as seen in this [example](https://medium.com/@VitalikButerin/quadratic-arithmetic-programs-from-zero-to-hero-f6d558cea649)) and output the result.

 example.py:


```
import sys

from pysnark.runtime import snark, PrivVal

@snark
def equation(x):
    return x*x*x+x+5

print("output:", equation(int(sys.argv[1])))
```



Getting the script:

In [3]:
!cd TrustedParty && wget https://raw.githubusercontent.com/giu176/NetworkSecurity-ZKP/main/example.py

--2022-08-25 09:09:50--  https://raw.githubusercontent.com/giu176/NetworkSecurity-ZKP/main/example.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 147 [text/plain]
Saving to: ‘example.py’

example.py          100%[===================>]     147  --.-KB/s    in 0s      

2022-08-25 09:09:50 (9.12 MB/s) - ‘example.py’ saved [147/147]



In [4]:
!cd TrustedParty && python example.py 3

*** Error loading backend pysnark.libsnark.backend: No module named 'libsnark'
*** Error loading backend pysnark.libsnark.backendgg: No module named 'libsnark'
output: 35
*** qaptools subroutines:
***    id: main function: main digest: 314adf18c4 #constraints: 5 *
*** generating master key material
*** new signature for function main, rebuilding keys
*** verification succeeded
***  prover keys/eqs:  pysnark_masterek pysnark_ek_main pysnark_eqs_main pysnark_schedule
***  prover data:      
***  verifier keys:    pysnark_masterpk pysnark_vk_main pysnark_schedule
***  verifier data:     pysnark_proof pysnark_values
***  verifier cmd:     qapver pysnark_masterpk pysnark_schedule pysnark_proof pysnark_values


Distributing files to the Prover:

In [5]:
!cp ./TrustedParty/example.py ./Prover/example.py
!cp ./TrustedParty/pysnark_schedule ./Prover/pysnark_schedule
!cp ./TrustedParty/pysnark_masterek ./Prover/pysnark_masterek
!cp ./TrustedParty/pysnark_ek_main ./Prover/pysnark_ek_main
!cp ./TrustedParty/pysnark_eqs_main ./Prover/pysnark_eqs_main
!cp ./TrustedParty/pysnark_masterpk ./Prover/pysnark_masterpk

Distributing files to the Verifier:

In [6]:
!cp ./TrustedParty/pysnark_schedule ./Verifier//pysnark_schedule
!cp ./TrustedParty/pysnark_masterpk ./Verifier/pysnark_masterpk
!cp ./TrustedParty/pysnark_vk_main ./Verifier/pysnark_vk_main

## Prover


---



In [7]:
!cd Prover && python example.py 3

*** Error loading backend pysnark.libsnark.backend: No module named 'libsnark'
*** Error loading backend pysnark.libsnark.backendgg: No module named 'libsnark'
output: 35
*** qaptools subroutines:
***    id: main function: main digest: 314adf18c4 #constraints: 5 *
*** verification keys missing, skipping verification
***  prover keys/eqs:  pysnark_masterek pysnark_ek_main pysnark_eqs_main pysnark_schedule
***  prover data:      
***  verifier keys:    pysnark_masterpk pysnark_vk_main pysnark_schedule
***  verifier data:     pysnark_proof pysnark_values
***  verifier cmd:     qapver pysnark_masterpk pysnark_schedule pysnark_proof pysnark_values


Distributing files to the Verifier:

In [8]:
!cp ./Prover/pysnark_proof ./Verifier/pysnark_proof
!cp ./Prover/pysnark_values ./Verifier/pysnark_values

## Verifier


---



In [9]:
!cd Verifier && python -m pysnark.qaptools.runqapver